- Preciso filtrar orders fulfillment
- Preciso filtrar orders aprovadas
- Preciso filtrar 'date_approved' em 'payments'. Se não houver desconsiderar

------

em cada resultado da lista de json existe um "shipping" com "id":
- preciso passar este id em: https://api.mercadolibre.com/shipments/{ship_id}
- procurar por 'fulfillment' em "logistic_type"
- se for fullfilment devo acrescentar dados ao df que calcula a qtd de produtos vendido

Em fulfillment aprovadas:
- Preciso saber a quantidade de vendas por id em determinado período (mesmo período da consulta ao estoque fulfillment, sempre findando no dia anterior a hoje)

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(ACCESS_TOKEN)

APP_USR-6628000663056293-111409-1babf87ade8fa0d80f9936a63144a99b-233632476


### Verificando dados da conta

In [3]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4521,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1518,"completed":32851,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

## Função

In [4]:
date_from = '2023-11-12'
date_to = '2023-11-13'

# URL base da API
base_url = "https://api.mercadolibre.com/orders/search"

# Parâmetros iniciais
params = {
    'seller': '233632476',
    # 'order.status': 'paid',
    'order.date_created.from': f'{date_from}T00:00:00.000-00:00',
    'order.date_created.to': f'{date_to}T00:00:00.000-00:00',
    'limit': 50,
    'offset': 0
}

# Token de autenticação
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

json_list = []

counter = 0

while True:
    response = requests.get(base_url, params=params, headers=headers)
    data = response.json()

    
    if 'results' in data:
        json_list.extend(data['results'])
    else:
        break
    
    # Verifique se há mais páginas
    if 'paging' in data:
        total_paging = data['paging'].get('total')
        if total_paging is None:
            break

        total_pages = math.ceil(total_paging / params['limit'])
        print(f'Total esperado de páginas: {counter}/{total_pages}')
        print(f'Offset atual: {params["offset"]}')

        counter += 1
        if params['offset'] >= total_paging:
            break

        params['offset'] += params['limit']
    else:
        break

print(f'Total de dados coletados: {len(json_list)}')

print(len(json_list))
json_list[0]

meta_cols = ['date_closed', 'pack_id', 'shipping', 'order_items']

df_payments = json_normalize(json_list, record_path=['payments'], meta=meta_cols)
df_payments.columns

# cols = ['date_approved','status','shipping','order_items']
cols = ['date_approved','status','shipping']
df_payments = df_payments[cols]

col = {'status':'payment_status'}
df_payments.rename(columns=col, inplace= True)

df_payments.head(1)

print(df_payments.shape)
df_payments = df_payments.dropna(subset=['date_approved'])
print(df_payments.shape)

df_payments['shipping_id'] = df_payments['shipping'].apply(lambda x: x['id'])
df_payments.head(1)

df_payments['shipping_id'] = df_payments['shipping_id'].astype(str).apply(lambda x: x.split('.')[0] if '.' in x else x)
df_payments = df_payments.drop('shipping', axis=1)

df_payments.head(1)
df_payments['shipping_id'].value_counts()

print(df_payments.shape)

df_payments = df_payments.drop_duplicates()

# Encontrando os índices das linhas com a data mais recente para cada shipping_id
indices_recentes = df_payments.groupby('shipping_id')['date_approved'].idxmax()

# Criando um novo DataFrame com base nos índices encontrados
df_payments = df_payments.loc[indices_recentes]

print(df_payments.shape)

# df_orders = json_normalize(json_list, record_path=['order_items'], meta=['date_closed', 'pack_id', 'status', 'shipping'])
df_orders = json_normalize(json_list, record_path=['order_items'], meta=['date_closed', 'pack_id', 'status', 'shipping'])

## pd.set_option('display.max_columns', None)
df_orders.columns

cols = ['quantity', 'item.id', 'item.title', 'item.category_id', 'item.variation_id',
       'item.seller_sku', 'date_closed', 'pack_id', 'status',
       'shipping']

df_orders = df_orders[cols]
df_orders.head(1)

df_orders['shipping_id'] = df_orders['shipping'].apply(lambda x: x['id'])
df_orders['shipping_id'] = df_orders['shipping_id'].astype(str).apply(lambda x: x.split('.')[0] if '.' in x else x)

df_orders = df_orders.drop('shipping', axis=1)
df_orders.head(1)

valores_unicos = df_orders['shipping_id'].value_counts()
print(valores_unicos)

df_resultado = pd.merge(df_orders, df_payments, on='shipping_id', how='left')
df_resultado.sample()

# pegando valores unicos para consula futura

uniq_shipping_id = df_resultado['shipping_id'].unique()


print(len(uniq_shipping_id))
print(df_orders.shape)
print(type(uniq_shipping_id))

json_shipments_list = []
success_count = 0
error_count = 0
counter = 0

for shipping_id in uniq_shipping_id:
    url = f"https://api.mercadolibre.com/shipments/{shipping_id}"

    payload = {}
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}'
    }

    try:
        print(f'Loop nº {counter}/{len(uniq_shipping_id)}: shipping_id = {shipping_id}')
        response = requests.request("GET", url, headers=headers, data=payload)
        response.raise_for_status()  # Verifica se a resposta tem um status de sucesso (código 2xx)

        json_shipments_list.append(response.json())
        success_count += 1
    except requests.exceptions.RequestException as e:
        # Lidar com erros de solicitação (por exemplo, conexão perdida, timeout)
        error_count += 1
        print(f"Erro na solicitação para shipping_id {shipping_id}: {str(e)}")
    counter +=1

print(f"Solicitações bem-sucedidas: {success_count}")
print(f"Erros de solicitação: {error_count}")

print(len(json_shipments_list))
print(type(json_shipments_list))

df = pd.DataFrame(json_shipments_list)

# df.head(1)
df.columns

df_orders = df_resultado.copy()
# Buscando envios do Full

df_fulfillment_ship = df[['id', 'logistic_type']]

df_fulfillment_ship = df_fulfillment_ship[df_fulfillment_ship['logistic_type'] == 'fulfillment']

print(df_orders.shape)
print(df_fulfillment_ship.shape)

print(df_fulfillment_ship['logistic_type'].value_counts())

df_fulfillment_ship['shipping_id'] = df_fulfillment_ship['id'].astype(str)
df_fulfillment_ship = df_fulfillment_ship.drop(['id'], axis=1)
df_fulfillment_ship.head(1)

# Unindo os DFs. Retorna apenas as linhas em df_orders onde shipping_id == id
df_res_fulfillment = pd.merge(df_orders, df_fulfillment_ship, left_on='shipping_id', right_on='shipping_id', how='inner')

df_res_fulfillment.shape

cols = ['quantity','item.id','item.seller_sku','date_closed', 'date_approved', 'pack_id', 'status', 'shipping_id', 'logistic_type']
df_order_res = df_res_fulfillment[cols]
df_order_res

conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)
df_codes.shape

# df_filtered = df_order_res.merge(df_codes, left_on='item.seller_sku', right_on='ml_sku', how='inner')
df_filtered = pd.merge(df_order_res, df_codes, left_on=['item.id', 'item.seller_sku'], right_on=['ml_code', 'ml_sku'], how='left')

df_filtered.head(3)

soma_por_ml_inventory_id = df_filtered.groupby('ml_inventory_id')['quantity'].sum().reset_index()
soma_por_ml_inventory_id.head(3)

conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM fulfillment_stock"

df_stock = pd.read_sql(sql_query, conn)

df_stock = df_stock.sort_values(by='created_at', ascending=False)
df_stock['data'] = df_stock['created_at'].dt.date

df_orders_f = df_filtered.copy()
df_orders_f.info()

df_orders_f['date_approved'] = pd.to_datetime(df_orders_f['date_approved'])
df_orders_f['data'] = df_orders_f['date_approved'].dt.date
df_orders_f = df_orders_f.drop(['date_closed', 'created_at', 'updated_at'], axis=1)

def filtrar_por_periodo(df, data_inicio, data_fim):
    return df[(df['data'] >= data_inicio) & (df['data'] <= data_fim)]

data_inicio = datetime(2023, 11, 10).date()  
data_fim = datetime(2023, 11, 13).date()     

# Filtrar DataFrames com base nas datas definidas
df_stock_filtrado = filtrar_por_periodo(df_stock, data_inicio, data_fim)
df_orders_filtrado = filtrar_por_periodo(df_orders_f, data_inicio, data_fim)

print(df_stock_filtrado.shape)
print(df_orders_filtrado.shape)

df_stock_filtrado.head(3)

## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock_filtrado = df_stock_filtrado.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock_filtrado.sort_values(by='data', ascending=False)
df_stock_filtrado.sample(4)

## Contando dias em que produto esteve disponível ##
days_available = df_stock_filtrado.groupby('ml_inventory_id')['has_stock'].sum().reset_index()
# # Renomeie a coluna resultante para 'days_available'
days_available = days_available.rename(columns={'has_stock': 'days_available'})

days_available.sample(5)

df_stock_filtrado = df_stock_filtrado.merge(days_available, on='ml_inventory_id', how='left')
df_stock_filtrado

total_sales_by_id = df_orders_filtrado.groupby('ml_inventory_id')['quantity'].sum().reset_index()
total_sales_by_id
# # Fusiona el DataFrame original con la suma total por ml_inventory_id
df_orders_filtrado = pd.merge(df_orders_filtrado, total_sales_by_id, on='ml_inventory_id', how='left')
df_orders_filtrado
# # # Renombra la columna resultante
df_orders_filtrado.rename(columns={'quantity_x': 'sales_quantity'}, inplace=True)
df_orders_filtrado.rename(columns={'quantity_y': 'total_sales_quantity'}, inplace=True)
df_orders_filtrado['sales_quantity'] = df_orders_filtrado['sales_quantity'].fillna(0)
df_orders_filtrado['total_sales_quantity'] = df_orders_filtrado['total_sales_quantity'].fillna(0)

df_orders_filtrado.sample(5)

df_stock_filtrado_ = df_stock_filtrado.copy()

indices_recentes = df_stock_filtrado_.groupby('ml_inventory_id')['data'].idxmax()
indices_recentes

df_stock_filtrado_ = df_stock_filtrado_.loc[indices_recentes]
df_stock_filtrado_ = df_stock_filtrado_.drop(['created_at', 'has_stock'], axis=1)

df_stock_filtrado_

df_orders_filtrado_ = df_orders_filtrado[['ml_inventory_id','item.id', 'ml_code', 'item.seller_sku' ,'ml_sku', 'date_approved', 'sales_quantity', 'total_sales_quantity']]
print(df_orders_filtrado_.shape)
df_orders_filtrado_ = df_orders_filtrado_.drop_duplicates()
print(df_orders_filtrado_.shape)
df_orders_filtrado_.sample()

_df_orders_filtrado_ = df_orders_filtrado_.copy()

indices_recentes = _df_orders_filtrado_.groupby('ml_inventory_id')['date_approved'].idxmax()
indices_recentes

_df_orders_filtrado_ = _df_orders_filtrado_.loc[indices_recentes]
# _df_orders_filtrado_ = _df_orders_filtrado_.drop(['created_at', 'has_stock'], axis=1)

print(df_orders_filtrado_.shape)
print(_df_orders_filtrado_.shape)

# df_calc = df_stock_filtrado_.merge(_df_orders_filtrado_, on='ml_inventory_id', how='left')
df_unido = pd.merge(df_stock_filtrado_, _df_orders_filtrado_, on='ml_inventory_id', how='left')
df_unido = df_unido.drop(['data', 'item.id', 'ml_code',	'item.seller_sku', 'ml_sku', 'date_approved'], axis=1)
df_unido['sales_quantity'] = df_unido['sales_quantity'].fillna(0)
df_unido['total_sales_quantity'] = df_unido['total_sales_quantity'].fillna(0)

df_unido.shape


days = 30

df_unido['media_prod_days_available'] = (df_unido['total_sales_quantity'] / df_unido['days_available'])
df_unido['media_prod_days_available'] = df_unido['media_prod_days_available'].fillna(0)

df_unido['30_send_fulfillment'] = np.ceil((df_unido['total_sales_quantity'] / df_unido['days_available'])* days - df_unido['available_quantity'])
df_unido['30_send_fulfillment'] = df_unido['30_send_fulfillment'].fillna(0)

df_unido['total_send_fulfillment'] = np.ceil((df_unido['total_sales_quantity'] / df_unido['days_available']) - df_unido['available_quantity'])
df_unido['total_send_fulfillment'] = df_unido['total_send_fulfillment'].fillna(0)


df_unido.to_csv('envio.csv')
df_unido.shape

Total esperado de páginas: 0/3
Offset atual: 0
Total esperado de páginas: 1/3
Offset atual: 50
Total esperado de páginas: 2/3
Offset atual: 100
Total esperado de páginas: 3/3
Offset atual: 150
Total de dados coletados: 138
138
(143, 3)
(139, 3)
(139, 3)
(135, 3)
shipping_id
42791017311    2
42789938478    2
42790974684    2
42791475484    1
42791369984    1
              ..
42790738144    1
42790623121    1
42790725304    1
42790722304    1
42791644361    1
Name: count, Length: 135, dtype: int64
135
(138, 10)
<class 'numpy.ndarray'>
Loop nº 0/135: shipping_id = 42789897426
Loop nº 1/135: shipping_id = 42789938478
Loop nº 2/135: shipping_id = 42789871609
Loop nº 3/135: shipping_id = 42789987690
Loop nº 4/135: shipping_id = 42789918733
Loop nº 5/135: shipping_id = 42789928457
Loop nº 6/135: shipping_id = 42790069634
Loop nº 7/135: shipping_id = 42789959153
Loop nº 8/135: shipping_id = 42790078138
Loop nº 9/135: shipping_id = 42790102298
Loop nº 10/135: shipping_id = 42790109020
Loop nº 1

C:\Users\couti\AppData\Local\Temp\ipykernel_17156\2752794348.py:198: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)
C:\Users\couti\AppData\Local\Temp\ipykernel_17156\2752794348.py:213: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   quantity         128 non-null    int64         
 1   item.id          128 non-null    object        
 2   item.seller_sku  128 non-null    object        
 3   date_closed      128 non-null    object        
 4   date_approved    128 non-null    object        
 5   pack_id          55 non-null     object        
 6   status           128 non-null    object        
 7   shipping_id      128 non-null    object        
 8   logistic_type    128 non-null    object        
 9   tiny_id          87 non-null     float64       
 10  tiny_sku         87 non-null     object        
 11  ml_inventory_id  87 non-null     object        
 12  ml_code          87 non-null     object        
 13  ml_sku           87 non-null     object        
 14  created_at       87 non-null     datetime6

(596, 8)

## Pegando dados de Vendas

A consulta a API do ML é feita com filtros para order.status = 'paid' e datas pré-determinadas

### Consultando API

In [4]:
date_from = '2023-11-12'
date_to = '2023-11-13'

In [5]:
# URL base da API
base_url = "https://api.mercadolibre.com/orders/search"

# Parâmetros iniciais
params = {
    'seller': '233632476',
    # 'order.status': 'paid',
    'order.date_created.from': f'{date_from}T00:00:00.000-00:00',
    'order.date_created.to': f'{date_to}T00:00:00.000-00:00',
    'limit': 50,
    'offset': 0
}

# Token de autenticação
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

json_list = []

In [6]:
counter = 0

while True:
    response = requests.get(base_url, params=params, headers=headers)
    data = response.json()

    
    if 'results' in data:
        json_list.extend(data['results'])
    else:
        break
    
    # Verifique se há mais páginas
    if 'paging' in data:
        total_paging = data['paging'].get('total')
        if total_paging is None:
            break

        total_pages = math.ceil(total_paging / params['limit'])
        print(f'Total esperado de páginas: {counter}/{total_pages}')
        print(f'Offset atual: {params["offset"]}')

        counter += 1
        if params['offset'] >= total_paging:
            break

        params['offset'] += params['limit']
    else:
        break

print(f'Total de dados coletados: {len(json_list)}')

Total esperado de páginas: 0/3
Offset atual: 0
Total esperado de páginas: 1/3
Offset atual: 50
Total esperado de páginas: 2/3
Offset atual: 100
Total esperado de páginas: 3/3
Offset atual: 150
Total de dados coletados: 138


In [24]:
print(len(json_list))
json_list[0]

138


{'payments': [{'reason': 'Kit 2 Pares De Baquetas D Bateria Liverpool Nirvana 5a 5b 7a',
   'status_code': None,
   'total_paid_amount': 40.28,
   'operation_type': 'regular_payment',
   'transaction_amount': 30.51,
   'transaction_amount_refunded': 0,
   'date_approved': '2023-11-11T20:03:37.000-04:00',
   'collector': {'id': 233632476},
   'coupon_id': None,
   'installments': 1,
   'authorization_code': None,
   'taxes_amount': 0,
   'id': 66680340315,
   'date_last_modified': '2023-11-12T10:12:33.000-04:00',
   'coupon_amount': 0,
   'available_actions': ['refund'],
   'shipping_cost': 9.77,
   'installment_amount': None,
   'date_created': '2023-11-11T20:02:06.000-04:00',
   'activation_uri': None,
   'overpaid_amount': 0,
   'card_id': None,
   'status_detail': 'accredited',
   'issuer_id': '12501',
   'payment_method_id': 'pix',
   'payment_type': 'bank_transfer',
   'deferred_period': None,
   'atm_transfer_reference': {'transaction_id': None, 'company_id': '1'},
   'site_id': 

### Pegando linhas onde existe 'date_approved'

In [25]:
meta_cols = ['date_closed', 'pack_id', 'shipping', 'order_items']

df_payments = json_normalize(json_list, record_path=['payments'], meta=meta_cols)
df_payments.columns


Index(['reason', 'status_code', 'total_paid_amount', 'operation_type',
       'transaction_amount', 'transaction_amount_refunded', 'date_approved',
       'coupon_id', 'installments', 'authorization_code', 'taxes_amount', 'id',
       'date_last_modified', 'coupon_amount', 'available_actions',
       'shipping_cost', 'installment_amount', 'date_created', 'activation_uri',
       'overpaid_amount', 'card_id', 'status_detail', 'issuer_id',
       'payment_method_id', 'payment_type', 'deferred_period', 'site_id',
       'payer_id', 'order_id', 'currency_id', 'status', 'transaction_order_id',
       'collector.id', 'atm_transfer_reference.transaction_id',
       'atm_transfer_reference.company_id', 'date_closed', 'pack_id',
       'shipping', 'order_items'],
      dtype='object')

In [26]:
df_payments.head(1)

,reason,status_code,total_paid_amount,operation_type,transaction_amount,transaction_amount_refunded,date_approved,coupon_id,installments,authorization_code,...,currency_id,status,transaction_order_id,collector.id,atm_transfer_reference.transaction_id,atm_transfer_reference.company_id,date_closed,pack_id,shipping,order_items
0,Kit 2 Pares De Baquetas D Bateria Liverpool Ni...,None,40.28,regular_payment,30.51,0.0,2023-11-11T20:03:37.000-04:00,None,1,None,...,BRL,approved,None,233632476,None,1,2023-11-11T20:04:39.000-04:00,None,{'id': 42789897426},"[{'item': {'id': 'MLB3696483496', 'title': 'Ki..."


In [27]:
# cols = ['date_approved','status','shipping','order_items']
cols = ['date_approved','status','shipping']
df_payments = df_payments[cols]

col = {'status':'payment_status'}
df_payments.rename(columns=col, inplace= True)

df_payments.head(1)

,date_approved,payment_status,shipping
0,2023-11-11T20:03:37.000-04:00,approved,{'id': 42789897426}


In [28]:
print(df_payments.shape)
df_payments = df_payments.dropna(subset=['date_approved'])
print(df_payments.shape)

(143, 3)
(139, 3)


In [29]:
df_payments.head(3)

,date_approved,payment_status,shipping
0,2023-11-11T20:03:37.000-04:00,approved,{'id': 42789897426}
1,2023-11-11T20:20:29.000-04:00,approved,{'id': 42789938478}
2,2023-11-11T20:20:29.000-04:00,approved,{'id': 42789938478}


### Pegando 'shipping.id' em 'payments'

Para verificar se uma ordem do ML foi do modo Full devo encontrar o shipping.id

In [30]:
df_payments['shipping_id'] = df_payments['shipping'].apply(lambda x: x['id'])
df_payments.head(1)

,date_approved,payment_status,shipping,shipping_id
0,2023-11-11T20:03:37.000-04:00,approved,{'id': 42789897426},42789897426


In [31]:
df_payments['shipping_id'] = df_payments['shipping_id'].astype(str).apply(lambda x: x.split('.')[0] if '.' in x else x)
df_payments = df_payments.drop('shipping', axis=1)

df_payments.head(1)

,date_approved,payment_status,shipping_id
0,2023-11-11T20:03:37.000-04:00,approved,42789897426


In [91]:
df_payments['shipping_id'].value_counts()

shipping_id
42791017311    2
42792317541    2
42791379825    1
42791335621    1
42791341553    1
              ..
42790699795    1
42790738144    1
42790623121    1
42790725304    1
42791644361    1
Name: count, Length: 135, dtype: int64

In [93]:
print(df_payments.shape)

df_payments = df_payments.drop_duplicates()

# Encontrando os índices das linhas com a data mais recente para cada shipping_id
indices_recentes = df_payments.groupby('shipping_id')['date_approved'].idxmax()

# Criando um novo DataFrame com base nos índices encontrados
df_payments = df_payments.loc[indices_recentes]

print(df_payments.shape)


(137, 3)
(135, 3)


### Pegando dados de produtos

In [94]:
# df_orders = json_normalize(json_list, record_path=['order_items'], meta=['date_closed', 'pack_id', 'status', 'shipping'])
df_orders = json_normalize(json_list, record_path=['order_items'], meta=['date_closed', 'pack_id', 'status', 'shipping'])

## pd.set_option('display.max_columns', None)
df_orders.columns

Index(['quantity', 'unit_price', 'full_unit_price', 'currency_id',
       'manufacturing_days', 'picked_quantity', 'sale_fee', 'listing_type_id',
       'base_exchange_rate', 'base_currency_id', 'bundle', 'element_id',
       'item.id', 'item.title', 'item.category_id', 'item.variation_id',
       'item.seller_custom_field', 'item.global_price', 'item.net_weight',
       'item.variation_attributes', 'item.warranty', 'item.condition',
       'item.seller_sku', 'requested_quantity.measure',
       'requested_quantity.value', 'date_closed', 'pack_id', 'status',
       'shipping'],
      dtype='object')

In [95]:
cols = ['quantity', 'item.id', 'item.title', 'item.category_id', 'item.variation_id',
       'item.seller_sku', 'date_closed', 'pack_id', 'status',
       'shipping']

df_orders = df_orders[cols]
df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping
0,1,MLB3696483496,Kit 2 Pares De Baquetas D Bateria Liverpool Ni...,MLB429603,1.785324e+11,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,None,paid,{'id': 42789897426}


### Pegando 'shipping.id' em 'orders'

Para verificar se uma ordem do ML foi do modo Full devo encontrar o shipping.id

In [96]:
df_orders['shipping_id'] = df_orders['shipping'].apply(lambda x: x['id'])
df_orders['shipping_id'] = df_orders['shipping_id'].astype(str).apply(lambda x: x.split('.')[0] if '.' in x else x)

df_orders = df_orders.drop('shipping', axis=1)
df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id
0,1,MLB3696483496,Kit 2 Pares De Baquetas D Bateria Liverpool Ni...,MLB429603,1.785324e+11,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,None,paid,42789897426


In [98]:
valores_unicos = df_orders['shipping_id'].value_counts()
print(valores_unicos)


shipping_id
42791017311    2
42789938478    2
42790974684    2
42791475484    1
42791369984    1
              ..
42790738144    1
42790623121    1
42790725304    1
42790722304    1
42791644361    1
Name: count, Length: 135, dtype: int64


In [99]:
x = df_orders[df_orders['shipping_id'] == '42791017311']
x

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id
70,1,MLB1992567302,1 Par Baquetas De Bateria Liverpool Nirvana 5a...,MLB429603,1.735853e+11,FULLNI5AM,2023-11-12T12:19:40.000-04:00,2000005030923241,paid,42791017311
71,1,MLB3900020030,Pandeiro Profissional Meia Lua Diversas Cores ...,MLB29319,NaN,FULLLIVERPOOLPML001A,2023-11-12T12:19:41.000-04:00,2000005030923241,paid,42791017311


## Combinando df_orders e df_payments para calcular produtos

In [100]:
df_orders.head(1)


,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id
0,1,MLB3696483496,Kit 2 Pares De Baquetas D Bateria Liverpool Ni...,MLB429603,1.785324e+11,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,None,paid,42789897426


In [101]:
df_payments.head(1)

,date_approved,payment_status,shipping_id
3,2023-11-11T20:40:43.000-04:00,approved,42789871609


In [102]:
print(df_orders.shape)
print(df_payments.shape)

(138, 10)
(135, 3)


In [106]:
df_orders.sample()

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id
107,1,MLB3427769549,Pandeiro Profissional Izzo Pele Preta 10 Corpo...,MLB29319,NaN,FULLIZZO6480,2023-11-12T16:39:40.000-04:00,2000005031728199,paid,42791795162


In [105]:
df_resultado = pd.merge(df_orders, df_payments, on='shipping_id', how='left')
df_resultado.sample()

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status
51,1,MLB2606294854,"2 Palheta Clarinete Vandoren Tradicional 1 1,5...",MLB278096,1.743839e+11,FULLVANDOREN2CR1025,2023-11-12T10:49:08.000-04:00,2000005030701027,paid,42790915668,2023-11-12T10:49:08.000-04:00,approved


In [107]:
df_resultado.shape

(138, 12)

In [112]:
df_resultado.sample(2)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status
11,1,MLB3829651804,Cabo P10 X P10 Santo Angelo Ninja 25ft - 7.62m,MLB72745,NaN,FULLP10NINJA25FT,2023-11-11T21:38:40.000-04:00,2000005029736075,paid,42790109020,2023-11-11T21:38:39.000-04:00,approved
38,1,MLB3474389897,Pele Remo Encore 14 Polegadas Ambassador Poros...,MLB45629,NaN,PELE10575,2023-11-12T09:26:34.000-04:00,None,paid,42790623121,2023-11-12T09:26:34.000-04:00,approved


In [113]:
x = df_resultado[df_resultado['shipping_id'] == '42790109020']
x

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status
11,1,MLB3829651804,Cabo P10 X P10 Santo Angelo Ninja 25ft - 7.62m,MLB72745,NaN,FULLP10NINJA25FT,2023-11-11T21:38:40.000-04:00,2000005029736075,paid,42790109020,2023-11-11T21:38:39.000-04:00,approved


In [114]:
df_resultado['shipping_id'].value_counts()

shipping_id
42791017311    2
42789938478    2
42790974684    2
42791475484    1
42791369984    1
              ..
42790738144    1
42790623121    1
42790725304    1
42790722304    1
42791644361    1
Name: count, Length: 135, dtype: int64

Pode haver mais de um produto por envio. Logo, shipping_id se repete no DF. Vamos remover as duplicatas para consultar o endpoint de envios 

## Consultando shipping_id

Devo verificar o tipo de envio, se for fulfillment devo sinalizar para futuramente filtrar o df criado no passo anterior

https://developers.mercadolivre.com.br/pt_br/gerenciamento-de-envios

```
curl -X OPTION -H 'Authorization: Bearer $ACCESS_TOKEN' https://api.mercadolibre.com/shipments/$SHIPMENT_ID
```

In [174]:
# pegando valores unicos para consula futura

uniq_shipping_id = df_resultado['shipping_id'].unique()


print(len(uniq_shipping_id))
print(df_orders.shape)
print(type(uniq_shipping_id))

135
(138, 10)
<class 'numpy.ndarray'>


In [116]:
json_shipments_list = []
success_count = 0
error_count = 0
counter = 0

for shipping_id in uniq_shipping_id:
    url = f"https://api.mercadolibre.com/shipments/{shipping_id}"

    payload = {}
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}'
    }

    try:
        print(f'Loop nº {counter}/{len(uniq_shipping_id)}: shipping_id = {shipping_id}')
        response = requests.request("GET", url, headers=headers, data=payload)
        response.raise_for_status()  # Verifica se a resposta tem um status de sucesso (código 2xx)

        json_shipments_list.append(response.json())
        success_count += 1
    except requests.exceptions.RequestException as e:
        # Lidar com erros de solicitação (por exemplo, conexão perdida, timeout)
        error_count += 1
        print(f"Erro na solicitação para shipping_id {shipping_id}: {str(e)}")
    counter +=1

print(f"Solicitações bem-sucedidas: {success_count}")
print(f"Erros de solicitação: {error_count}")


Loop nº 0/135: shipping_id = 42789897426
Loop nº 1/135: shipping_id = 42789938478
Loop nº 2/135: shipping_id = 42789871609
Loop nº 3/135: shipping_id = 42789987690
Loop nº 4/135: shipping_id = 42789918733
Loop nº 5/135: shipping_id = 42789928457
Loop nº 6/135: shipping_id = 42790069634
Loop nº 7/135: shipping_id = 42789959153
Loop nº 8/135: shipping_id = 42790078138
Loop nº 9/135: shipping_id = 42790102298
Loop nº 10/135: shipping_id = 42790109020
Loop nº 11/135: shipping_id = 42790224756
Loop nº 12/135: shipping_id = 42790244292
Loop nº 13/135: shipping_id = 42790270444
Loop nº 14/135: shipping_id = 42790191651
Loop nº 15/135: shipping_id = 42790320332
Loop nº 16/135: shipping_id = 42790344728
Loop nº 17/135: shipping_id = 42790345750
Loop nº 18/135: shipping_id = 42790249225
Loop nº 19/135: shipping_id = 42790366165
Loop nº 20/135: shipping_id = 42790428483
Loop nº 21/135: shipping_id = 42790443205
Loop nº 22/135: shipping_id = 42790454841
Loop nº 23/135: shipping_id = 42790578166
Lo

In [175]:
print(len(json_shipments_list))
print(type(json_shipments_list))

135
<class 'list'>


In [176]:
json_shipments_list

[{'substatus_history': [{'date': '2023-11-11T20:04:42.000-04:00',
    'substatus': 'in_warehouse',
    'status': 'ready_to_ship'},
   {'date': '2023-11-11T20:49:29.834-04:00',
    'substatus': 'ready_to_pack',
    'status': 'ready_to_ship'},
   {'date': '2023-11-11T21:20:13.034-04:00',
    'substatus': 'packed',
    'status': 'ready_to_ship'},
   {'date': '2023-11-11T21:22:26.000-04:00',
    'substatus': 'in_packing_list',
    'status': 'ready_to_ship'}],
  'snapshot_packing': {'snapshot_id': '6b07e2aa-89c8-47fd-b757-70e99772c5c9',
   'pack_hash': '1'},
  'receiver_id': 240294416,
  'base_cost': 11.5,
  'status_history': {'date_shipped': '2023-11-12T01:40:39.788-04:00',
   'date_returned': None,
   'date_delivered': '2023-11-12T10:12:28.000-04:00',
   'date_first_visit': '2023-11-12T10:12:28.000-04:00',
   'date_not_delivered': None,
   'date_cancelled': None,
   'date_handling': '2023-11-11T20:04:42.000-04:00',
   'date_ready_to_ship': '2023-11-11T20:04:42.000-04:00'},
  'type': 'forw

In [177]:
df = pd.DataFrame(json_shipments_list)

# df.head(1)
df.columns


Index(['substatus_history', 'snapshot_packing', 'receiver_id', 'base_cost',
       'status_history', 'type', 'return_details', 'sender_id', 'mode',
       'order_cost', 'priority_class', 'service_id', 'shipping_items',
       'tracking_number', 'cost_components', 'id', 'tracking_method',
       'last_updated', 'items_types', 'comments', 'substatus', 'date_created',
       'date_first_printed', 'created_by', 'application_id', 'shipping_option',
       'tags', 'sender_address', 'sibling', 'return_tracking_number',
       'site_id', 'carrier_info', 'market_place', 'receiver_address',
       'customer_id', 'order_id', 'quotation', 'status', 'logistic_type'],
      dtype='object')

In [178]:
df.head(1)

,substatus_history,snapshot_packing,receiver_id,base_cost,status_history,type,return_details,sender_id,mode,order_cost,...,return_tracking_number,site_id,carrier_info,market_place,receiver_address,customer_id,order_id,quotation,status,logistic_type
0,"[{'date': '2023-11-11T20:04:42.000-04:00', 'su...",{'snapshot_id': '6b07e2aa-89c8-47fd-b757-70e99...,240294416,11.5,{'date_shipped': '2023-11-12T01:40:39.788-04:0...,forward,None,233632476,me2,30.51,...,None,MLB,None,MELI,"{'country': {'id': 'BR', 'name': 'Brasil'}, 'a...",None,2000006871971774,None,delivered,fulfillment


In [123]:
print(df['logistic_type'].value_counts())

logistic_type
fulfillment      103
cross_docking     21
self_service       9
default            2
Name: count, dtype: int64


In [179]:
df_orders = df_resultado.copy()
# Buscando envios do Full

df_fulfillment_ship = df[['id', 'logistic_type']]

df_fulfillment_ship = df_fulfillment_ship[df_fulfillment_ship['logistic_type'] == 'fulfillment']

print(df_orders.shape)
print(df_fulfillment_ship.shape)

print(df_fulfillment_ship['logistic_type'].value_counts())

(138, 12)
(103, 2)
logistic_type
fulfillment    103
Name: count, dtype: int64


In [202]:
df_orders.head(1)

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status
0,1,MLB3696483496,Kit 2 Pares De Baquetas D Bateria Liverpool Ni...,MLB429603,1.785324e+11,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,None,paid,42789897426,2023-11-11T20:03:37.000-04:00,approved


In [181]:
df_fulfillment_ship.head(1)

,id,logistic_type
0,42789897426,fulfillment


In [182]:
df_fulfillment_ship['shipping_id'] = df_fulfillment_ship['id'].astype(str)
df_fulfillment_ship = df_fulfillment_ship.drop(['id'], axis=1)
df_fulfillment_ship.head(1)

,logistic_type,shipping_id
0,fulfillment,42789897426


In [183]:
df_orders['shipping_id'].value_counts()

shipping_id
42791017311    2
42789938478    2
42790974684    2
42791475484    1
42791369984    1
              ..
42790738144    1
42790623121    1
42790725304    1
42790722304    1
42791644361    1
Name: count, Length: 135, dtype: int64

In [184]:
df_fulfillment_ship['shipping_id'].value_counts()

shipping_id
42789897426    1
42791342860    1
42791454807    1
42791404329    1
42791380335    1
              ..
42790701701    1
42790699795    1
42790738144    1
42790725304    1
42791644361    1
Name: count, Length: 103, dtype: int64

In [203]:
# Unindo os DFs. Retorna apenas as linhas em df_orders onde shipping_id == id
df_res_fulfillment = pd.merge(df_orders, df_fulfillment_ship, left_on='shipping_id', right_on='shipping_id', how='inner')

df_res_fulfillment.shape

(106, 13)

In [204]:
df_res_fulfillment.sample()

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status,logistic_type
46,1,MLB1992889189,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,MLB429603,1.735513e+11,FULLLIVERPOOL3NI5AM,2023-11-12T11:10:25.000-04:00,None,paid,42790966836,2023-11-12T11:10:25.000-04:00,approved,fulfillment


In [205]:
df_res_fulfillment['shipping_id'].value_counts()

shipping_id
42789938478    2
42791017311    2
42790974684    2
42791342860    1
42791454807    1
              ..
42790701701    1
42790699795    1
42790738144    1
42790725304    1
42791644361    1
Name: count, Length: 103, dtype: int64

In [206]:
x = df_res_fulfillment[df_res_fulfillment['shipping_id'] == '42789938478']
x

,quantity,item.id,item.title,item.category_id,item.variation_id,item.seller_sku,date_closed,pack_id,status,shipping_id,date_approved,payment_status,logistic_type
1,1,MLB3337214843,Kit 6 Pares De Baquetas D Bateria Liverpool Ni...,MLB429603,1.784974e+11,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11T20:20:30.000-04:00,2000005029535641,paid,42789938478,2023-11-11T20:20:29.000-04:00,approved,fulfillment
2,1,MLB3337214843,Kit 6 Pares De Baquetas D Bateria Liverpool Ni...,MLB429603,1.784974e+11,FULLLIVERPOOL6NI7AM,2023-11-11T20:20:30.000-04:00,2000005029535641,paid,42789938478,2023-11-11T20:20:29.000-04:00,approved,fulfillment


In [207]:
df_res_fulfillment.columns

Index(['quantity', 'item.id', 'item.title', 'item.category_id',
       'item.variation_id', 'item.seller_sku', 'date_closed', 'pack_id',
       'status', 'shipping_id', 'date_approved', 'payment_status',
       'logistic_type'],
      dtype='object')

In [209]:
cols = ['quantity','item.id','item.seller_sku','date_closed', 'date_approved', 'pack_id', 'status', 'shipping_id', 'logistic_type']
df_order_res = df_res_fulfillment[cols]
df_order_res

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type
0,1,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,2023-11-11T20:03:37.000-04:00,None,paid,42789897426,fulfillment
1,1,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11T20:20:30.000-04:00,2023-11-11T20:20:29.000-04:00,2000005029535641,paid,42789938478,fulfillment
2,1,MLB3337214843,FULLLIVERPOOL6NI7AM,2023-11-11T20:20:30.000-04:00,2023-11-11T20:20:29.000-04:00,2000005029535641,paid,42789938478,fulfillment
3,1,MLB1992889189,FULLLIVERPOOL3NI5AM,2023-11-11T20:40:43.000-04:00,2023-11-11T20:40:43.000-04:00,None,paid,42789871609,fulfillment
4,1,MLB3438642951,FULLERNIEBALLP04049,2023-11-11T20:42:12.000-04:00,2023-11-11T20:42:11.000-04:00,2000005029592131,paid,42789987690,fulfillment
...,...,...,...,...,...,...,...,...,...
101,1,MLB3900020030,FULLLIVERPOOLPML001A,2023-11-12T19:51:40.000-04:00,2023-11-12T19:51:17.000-04:00,None,paid,42792200697,fulfillment
102,1,MLB1991060554,FULLBASSOCLA01,2023-11-12T20:08:40.000-04:00,2023-11-12T20:08:40.000-04:00,None,paid,42792367812,fulfillment
103,1,MLB3860760526,FULLDOLPHIN6408,2023-11-12T20:35:42.000-04:00,2023-11-12T20:34:41.000-04:00,None,paid,42792317541,fulfillment
104,1,MLB3780803056,FULLBASSOCLA09,2023-11-12T20:47:15.000-04:00,2023-11-12T20:47:15.000-04:00,None,paid,42792478070,fulfillment


In [210]:
x = df_order_res[df_order_res['shipping_id'] == '42789938478']
x

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type
1,1,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11T20:20:30.000-04:00,2023-11-11T20:20:29.000-04:00,2000005029535641,paid,42789938478,fulfillment
2,1,MLB3337214843,FULLLIVERPOOL6NI7AM,2023-11-11T20:20:30.000-04:00,2023-11-11T20:20:29.000-04:00,2000005029535641,paid,42789938478,fulfillment


## Pegando códigos para comparar os produtos que queremos

In [211]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)
df_codes.shape

C:\Users\couti\AppData\Local\Temp\ipykernel_12708\2578135159.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


(878, 7)

In [212]:
df_codes.columns
df_order_res.columns

Index(['quantity', 'item.id', 'item.seller_sku', 'date_closed',
       'date_approved', 'pack_id', 'status', 'shipping_id', 'logistic_type'],
      dtype='object')

In [213]:
df_codes.head(3)

,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,509517168,7899028808537,JFGN34621,MLB924922735,FULLP10NINJA10FTC,2023-11-10 16:32:30.451360,None
1,598796832,7899028824407,ERIM51807,MLB949788598,P10LNINJA10FT,2023-11-10 16:32:30.451360,None
2,565287467,7899028808773,HVCS52763,MLB950214971,FULLLWNINJA10FT,2023-11-10 16:32:30.451360,None


In [214]:
df_order_res.head(1)

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type
0,1,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,2023-11-11T20:03:37.000-04:00,None,paid,42789897426,fulfillment


In [215]:
x = df_codes[df_codes['ml_code'] == 'MLB3337214843']
x

,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
805,698412654,7897937421014,NaN,MLB3337214843,FULLLIVERPOOL6NI5AM,2023-11-10 16:32:30.451360,None
806,735947207,7897937421021,NaN,MLB3337214843,FULLLIVERPOOL6NI5BM,2023-11-10 16:32:30.451360,None
807,747848159,7897937421007,NaN,MLB3337214843,FULLLIVERPOOL6NI7AM,2023-11-10 16:32:30.451360,None
808,698412654,7897937421014,NaN,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-10 16:32:30.451360,None
809,735947207,7897937421021,NaN,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-10 16:32:30.451360,None
810,747848159,7897937421007,NaN,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-10 16:32:30.451360,None
811,698412654,7897937421014,NaN,MLB3337214843,FULLLIVERPOOL3NI5AM3NI5BM,2023-11-10 16:32:30.451360,None
812,735947207,7897937421021,NaN,MLB3337214843,FULLLIVERPOOL3NI5AM3NI5BM,2023-11-10 16:32:30.451360,None
813,735947207,7897937421021,NaN,MLB3337214843,FULLLIVERPOOL3NI5BM3NI7AM,2023-11-10 16:32:30.451360,None
814,747848159,7897937421007,NaN,MLB3337214843,FULLLIVERPOOL3NI5BM3NI7AM,2023-11-10 16:32:30.451360,None


## Comparando os dois DFs

In [216]:
# df_filtered = df_order_res.merge(df_codes, left_on='item.seller_sku', right_on='ml_sku', how='inner')
df_filtered = pd.merge(df_order_res, df_codes, left_on=['item.id', 'item.seller_sku'], right_on=['ml_code', 'ml_sku'], how='left')

df_filtered.head(3)

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,1,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,2023-11-11T20:03:37.000-04:00,None,paid,42789897426,fulfillment,698412654.0,7897937421014,NaN,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-10 16:32:30.451360,None
1,1,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11T20:20:30.000-04:00,2023-11-11T20:20:29.000-04:00,2000005029535641,paid,42789938478,fulfillment,698412654.0,7897937421014,NaN,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-10 16:32:30.451360,None
2,1,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11T20:20:30.000-04:00,2023-11-11T20:20:29.000-04:00,2000005029535641,paid,42789938478,fulfillment,735947207.0,7897937421021,NaN,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-10 16:32:30.451360,None


In [217]:
print(df_filtered.shape)

(128, 16)


In [218]:
print(df_filtered['quantity'].value_counts(),'\n')

# print(df_filtered['item.seller_sku'].value_counts(),'\n')

print(df_filtered['status'].value_counts(),'\n')

# print(df_filtered['shipping_id'].value_counts(),'\n')

print(df_filtered['logistic_type'].value_counts(),'\n')

# print(df_filtered['tiny_id'].value_counts(),'\n')

# print(df_filtered['ml_inventory_id'].value_counts(),'\n')

print(df_filtered['ml_code'].value_counts(),'\n')


quantity
1    117
2      6
4      2
3      2
6      1
Name: count, dtype: int64 

status
paid         126
cancelled      2
Name: count, dtype: int64 

logistic_type
fulfillment    128
Name: count, dtype: int64 

ml_code
MLB1992889189    11
MLB2001037689     8
MLB3337214843     7
MLB2606294854     6
MLB1813063328     5
MLB1813530858     4
MLB2000969658     4
MLB1991060554     4
MLB3339660079     4
MLB1992567302     3
MLB2770292345     2
MLB1556975175     2
MLB1556962508     2
MLB3780803056     2
MLB1557014758     2
MLB2022465762     2
MLB924922735      2
MLB2024265635     2
MLB2024002438     2
MLB2075516288     1
MLB1399101853     1
MLB1476093449     1
MLB2216644379     1
MLB3696483496     1
MLB2001207578     1
MLB3778540120     1
MLB1662862664     1
MLB1403763907     1
MLB1847067851     1
MLB2004848109     1
MLB1464478537     1
MLB2001122140     1
Name: count, dtype: int64 



In [219]:
# print(df_filtered['tiny_id'].value_counts(),'\n')

In [220]:
# dfx = df_filtered[df_filtered['tiny_id'] == 747848159]
# dfx.head(5)

## Contagem de produtos vendidos

In [221]:
df_filtered.head(1)

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,1,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,2023-11-11T20:03:37.000-04:00,None,paid,42789897426,fulfillment,698412654.0,7897937421014,NaN,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-10 16:32:30.451360,None


In [222]:
soma_por_ml_inventory_id = df_filtered.groupby('ml_inventory_id')['quantity'].sum().reset_index()
soma_por_ml_inventory_id.head(3)

,ml_inventory_id,quantity
0,CDBO11155,1
1,CIKE94068,1
2,CRMK90073,1


In [223]:
soma_por_ml_code = df_filtered.groupby('ml_code')['quantity'].sum().reset_index()
soma_por_ml_code.head(3)

,ml_code,quantity
0,MLB1399101853,1
1,MLB1403763907,1
2,MLB1464478537,1


In [224]:
soma_por_tiny_id = df_filtered.groupby('tiny_id')['quantity'].sum().reset_index()
soma_por_tiny_id.head(3)

,tiny_id,quantity
0,509517168.0,2
1,509978212.0,1
2,510044277.0,4


In [225]:
soma_por_sku = df_filtered.groupby('ml_sku')['quantity'].sum().reset_index()
soma_por_sku.head(3)

,ml_sku,quantity
0,FULLBASSOCLA01,4
1,FULLBASSOCLA09,2
2,FULLBASSOCLA13KIT,4


In [226]:
x = df_filtered[df_filtered['ml_sku'] == 'FULLBASSOCLA13KIT']
x

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
87,1,MLB2000969658,FULLBASSOCLA13KIT,2023-11-12T13:46:17.000-04:00,2023-11-12T13:46:17.000-04:00,2000005031193981,paid,42791234849,fulfillment,729129794.0,7898942136160,WEPI04061,MLB2000969658,FULLBASSOCLA13KIT,2023-11-10 16:32:30.451360,None
88,1,MLB2000969658,FULLBASSOCLA13KIT,2023-11-12T13:46:17.000-04:00,2023-11-12T13:46:17.000-04:00,2000005031193981,paid,42791234849,fulfillment,565665071.0,Palheta,WEPI04061,MLB2000969658,FULLBASSOCLA13KIT,2023-11-10 16:32:30.451360,None
89,1,MLB2000969658,FULLBASSOCLA13KIT,2023-11-12T13:46:17.000-04:00,2023-11-12T13:46:17.000-04:00,2000005031193981,paid,42791234849,fulfillment,565663877.0,PPalheta,WEPI04061,MLB2000969658,FULLBASSOCLA13KIT,2023-11-10 16:32:30.451360,None
90,1,MLB2000969658,FULLBASSOCLA13KIT,2023-11-12T13:46:17.000-04:00,2023-11-12T13:46:17.000-04:00,2000005031193981,paid,42791234849,fulfillment,746248920.0,RH-01BK,WEPI04061,MLB2000969658,FULLBASSOCLA13KIT,2023-11-10 16:32:30.451360,None


In [227]:
x = df_filtered[df_filtered['tiny_id'] == 509517168.0]
x

,quantity,item.id,item.seller_sku,date_closed,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
12,1,MLB924922735,FULLP10NINJA10FTC,2023-11-11T21:35:22.000-04:00,2023-11-11T21:35:22.000-04:00,2000005029717373,paid,42790102298,fulfillment,509517168.0,7899028808537,JFGN34621,MLB924922735,FULLP10NINJA10FTC,2023-11-10 16:32:30.451360,None
61,1,MLB924922735,FULLP10NINJA10FTC,2023-11-12T11:22:24.000-04:00,2023-11-12T11:22:24.000-04:00,2000005030785299,paid,42790877583,fulfillment,509517168.0,7899028808537,JFGN34621,MLB924922735,FULLP10NINJA10FTC,2023-11-10 16:32:30.451360,None


In [228]:
print(df_filtered.shape)
print(soma_por_ml_inventory_id.shape)
print(soma_por_ml_code.shape)
print(soma_por_tiny_id.shape)

(128, 16)
(35, 2)
(32, 2)
(41, 2)


## Calculando estoque

### Pegando estoque do DB

In [229]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM fulfillment_stock"

df_stock = pd.read_sql(sql_query, conn)

df_stock = df_stock.sort_values(by='created_at', ascending=False)
df_stock['data'] = df_stock['created_at'].dt.date


C:\Users\couti\AppData\Local\Temp\ipykernel_12708\1772980314.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


In [230]:
df_stock

,ml_inventory_id,available_quantity,created_at,data
19667,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13
19275,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13
19273,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13
19272,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13
19271,GLMK71067,0,2023-11-13 12:31:28.615487,2023-11-13
...,...,...,...,...
18081,EDGX70669,0,2023-10-10 01:00:00.000000,2023-10-10
18082,KYGE71098,0,2023-10-10 01:00:00.000000,2023-10-10
18083,LDBL69476,0,2023-10-10 01:00:00.000000,2023-10-10
18084,PUCU70522,0,2023-10-10 01:00:00.000000,2023-10-10


In [232]:
df_orders_f = df_filtered.copy()
df_orders_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   quantity         128 non-null    int64         
 1   item.id          128 non-null    object        
 2   item.seller_sku  128 non-null    object        
 3   date_closed      128 non-null    object        
 4   date_approved    128 non-null    object        
 5   pack_id          55 non-null     object        
 6   status           128 non-null    object        
 7   shipping_id      128 non-null    object        
 8   logistic_type    128 non-null    object        
 9   tiny_id          87 non-null     float64       
 10  tiny_sku         87 non-null     object        
 11  ml_inventory_id  87 non-null     object        
 12  ml_code          87 non-null     object        
 13  ml_sku           87 non-null     object        
 14  created_at       87 non-null     datetime6

In [233]:
df_orders_f['date_approved'] = pd.to_datetime(df_orders_f['date_approved'])
df_orders_f['data'] = df_orders_f['date_approved'].dt.date
df_orders_f = df_orders_f.drop(['date_closed', 'created_at', 'updated_at'], axis=1)


In [172]:
df_orders_f.head(1)

,quantity,item.id,item.seller_sku,date_closed,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,created_at,updated_at
0,1,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-11T20:04:39.000-04:00,None,paid,42789897426,fulfillment,698412654.0,7897937421014,NaN,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-10 16:32:30.451360,None


In [234]:
df_stock.head(1)

,ml_inventory_id,available_quantity,created_at,data
19667,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13


### Filtrando o período de consulta

In [468]:
def filtrar_por_periodo(df, data_inicio, data_fim):
    return df[(df['data'] >= data_inicio) & (df['data'] <= data_fim)]

data_inicio = datetime(2023, 11, 10).date()  
data_fim = datetime(2023, 11, 13).date()     

# Filtrar DataFrames com base nas datas definidas
df_stock_filtrado = filtrar_por_periodo(df_stock, data_inicio, data_fim)
df_orders_filtrado = filtrar_por_periodo(df_orders_f, data_inicio, data_fim)

print(df_stock_filtrado.shape)
print(df_orders_filtrado.shape)

df_stock_filtrado.head(3)

(1192, 4)
(128, 14)


,ml_inventory_id,available_quantity,created_at,data
19667,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13
19275,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13
19273,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13


In [469]:
df_orders_filtrado.head(3)

,quantity,item.id,item.seller_sku,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,data
0,1,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-11 20:03:37-04:00,None,paid,42789897426,fulfillment,698412654.0,7897937421014,NaN,MLB3696483496,FULLLIVERPOOL2NI5AM,2023-11-11
1,1,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11 20:20:29-04:00,2000005029535641,paid,42789938478,fulfillment,698412654.0,7897937421014,NaN,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11
2,1,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11 20:20:29-04:00,2000005029535641,paid,42789938478,fulfillment,735947207.0,7897937421021,NaN,MLB3337214843,FULLLIVERPOOL2NI5AM2NI5BM2NI7AM,2023-11-11


### Verificando se existe no estoque

In [470]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_stock_filtrado = df_stock_filtrado.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_stock_filtrado.sort_values(by='data', ascending=False)
df_stock_filtrado.sample(4)

,ml_inventory_id,available_quantity,created_at,data,has_stock
19297,WRRA78397,0,2023-11-13 12:31:28.615487,2023-11-13,False
18566,MTUF40103,0,2023-11-10 20:22:52.890804,2023-11-10,False
18838,RLWO47035,0,2023-11-10 20:22:52.890804,2023-11-10,False
19623,IYUQ81169,6,2023-11-13 12:31:28.615487,2023-11-13,True


### Contando os dias em que esteve disponível

In [471]:
## Contando dias em que produto esteve disponível ##
days_available = df_stock_filtrado.groupby('ml_inventory_id')['has_stock'].sum().reset_index()
# # Renomeie a coluna resultante para 'days_available'
days_available = days_available.rename(columns={'has_stock': 'days_available'})

days_available.sample(5)

,ml_inventory_id,days_available
358,PNZZ04589,0
500,WCNJ53438,2
524,WTQW89994,0
365,PQCQ70522,0
45,CLMF99895,2


In [472]:
df_stock_filtrado = df_stock_filtrado.merge(days_available, on='ml_inventory_id', how='left')
df_stock_filtrado

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
0,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
1,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
2,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
3,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
4,GLMK71067,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
...,...,...,...,...,...,...
1187,IVOT73839,0,2023-11-10 20:22:52.890804,2023-11-10,False,0
1188,WPGI76636,0,2023-11-10 20:22:52.890804,2023-11-10,False,0
1189,UCDR32919,14,2023-11-10 20:22:52.890804,2023-11-10,True,2
1190,GUSW29256,2,2023-11-10 20:22:52.890804,2023-11-10,True,2


In [473]:
1192 / 596

2.0

### Calculando total de produtos por período

In [474]:
# total_quantity_by_id = df_stock_filtrado.groupby('ml_inventory_id')['available_quantity'].sum().reset_index()
# total_quantity_by_id
# # # Fusiona el DataFrame original con la suma total por ml_inventory_id
# df_stock_filtrado = pd.merge(df_stock_filtrado, total_quantity_by_id, on='ml_inventory_id', how='left')

# # # Renombra la columna resultante
# df_stock_filtrado.rename(columns={'available_quantity_x': 'available_quantity'}, inplace=True)
# df_stock_filtrado.rename(columns={'available_quantity_y': 'total_available_quantity'}, inplace=True)
# df_stock_filtrado

In [475]:
x = df_stock_filtrado[df_stock_filtrado['ml_inventory_id'] == 'UCDR32919']
x

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
593,UCDR32919,13,2023-11-13 12:31:28.615487,2023-11-13,True,2
1189,UCDR32919,14,2023-11-10 20:22:52.890804,2023-11-10,True,2


In [476]:
df_stock_filtrado.shape

(1192, 6)

### Calculando total de vendas por produto por período

In [477]:
total_sales_by_id = df_orders_filtrado.groupby('ml_inventory_id')['quantity'].sum().reset_index()
total_sales_by_id
# # Fusiona el DataFrame original con la suma total por ml_inventory_id
df_orders_filtrado = pd.merge(df_orders_filtrado, total_sales_by_id, on='ml_inventory_id', how='left')
df_orders_filtrado
# # # Renombra la columna resultante
df_orders_filtrado.rename(columns={'quantity_x': 'sales_quantity'}, inplace=True)
df_orders_filtrado.rename(columns={'quantity_y': 'total_sales_quantity'}, inplace=True)
df_orders_filtrado['sales_quantity'] = df_orders_filtrado['sales_quantity'].fillna(0)
df_orders_filtrado['total_sales_quantity'] = df_orders_filtrado['total_sales_quantity'].fillna(0)

df_orders_filtrado.sample(5)

,sales_quantity,item.id,item.seller_sku,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,data,total_sales_quantity
109,1,MLB3427769549,FULLIZZO6480,2023-11-12 17:54:52-04:00,None,paid,42791987820,fulfillment,NaN,NaN,NaN,NaN,NaN,2023-11-12,0.0
65,1,MLB2024002438,FULLERNIEBALLP02150,2023-11-12 11:52:51-04:00,2000005030863445,paid,42791065574,fulfillment,730856842.0,749699121503,XYRN72687,MLB2024002438,FULLERNIEBALLP02150,2023-11-12,1.0
54,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 10:54:09-04:00,None,paid,42790812637,fulfillment,565663877.0,PPalheta,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
114,1,MLB1992889189,FULLLIVERPOOL3NI5AM,2023-11-12 18:34:00-04:00,None,paid,42791980241,fulfillment,698412654.0,7897937421014,FSNB97805,MLB1992889189,FULLLIVERPOOL3NI5AM,2023-11-12,4.0
101,1,MLB2022465762,FULLKP02004,2023-11-12 15:33:07-04:00,2000005031521835,paid,42791507387,fulfillment,693267123.0,749699120049,CDBO11155,MLB2022465762,FULLKP02004,2023-11-12,1.0


In [478]:
x = df_orders_filtrado[df_orders_filtrado['ml_inventory_id'] == 'DZVJ05686']
x

,sales_quantity,item.id,item.seller_sku,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,data,total_sales_quantity
79,1,MLB1556975175,FULLTAGIMA21KCOMCAPA,2023-11-12 13:12:41-04:00,None,paid,42791266148,fulfillment,510044277.0,7898563976213,DZVJ05686,MLB1556975175,FULLTAGIMA21KCOMCAPA,2023-11-12,2.0
80,1,MLB1556975175,FULLTAGIMA21KCOMCAPA,2023-11-12 13:12:41-04:00,None,paid,42791266148,fulfillment,759523162.0,A90,DZVJ05686,MLB1556975175,FULLTAGIMA21KCOMCAPA,2023-11-12,2.0


In [479]:
x = df_orders_filtrado[df_orders_filtrado['ml_inventory_id'] == 'YZGH03888']
x

,sales_quantity,item.id,item.seller_sku,date_approved,pack_id,status,shipping_id,logistic_type,tiny_id,tiny_sku,ml_inventory_id,ml_code,ml_sku,data,total_sales_quantity
52,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 10:54:09-04:00,None,paid,42790812637,fulfillment,729129899.0,7898529535546,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
53,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 10:54:09-04:00,None,paid,42790812637,fulfillment,565665071.0,Palheta,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
54,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 10:54:09-04:00,None,paid,42790812637,fulfillment,565663877.0,PPalheta,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
55,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 10:54:09-04:00,None,paid,42790812637,fulfillment,746249012.0,RH-01SV,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
73,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 12:26:54-04:00,None,paid,42791037091,fulfillment,729129899.0,7898529535546,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
74,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 12:26:54-04:00,None,paid,42791037091,fulfillment,565665071.0,Palheta,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
75,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 12:26:54-04:00,None,paid,42791037091,fulfillment,565663877.0,PPalheta,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0
76,1,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12 12:26:54-04:00,None,paid,42791037091,fulfillment,746249012.0,RH-01SV,YZGH03888,MLB2001037689,FULLBASSOCLA25KIT,2023-11-12,8.0


### Unindo DFs por 'ml_inventory_id'

In [480]:
df_stock_filtrado.sample()

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
1025,IYUQ71257,0,2023-11-10 20:22:52.890804,2023-11-10,False,0


In [481]:
x = df_stock_filtrado[df_stock_filtrado['ml_inventory_id'] == 'DGCL82617']
x

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
434,DGCL82617,20,2023-11-13 12:31:28.615487,2023-11-13,True,2
1030,DGCL82617,24,2023-11-10 20:22:52.890804,2023-11-10,True,2


In [482]:
df_stock_filtrado_ = df_stock_filtrado.copy()

indices_recentes = df_stock_filtrado_.groupby('ml_inventory_id')['data'].idxmax()
indices_recentes

df_stock_filtrado_ = df_stock_filtrado_.loc[indices_recentes]
df_stock_filtrado_ = df_stock_filtrado_.drop(['created_at', 'has_stock'], axis=1)

df_stock_filtrado_

,ml_inventory_id,available_quantity,data,days_available
189,ABCB20467,0,2023-11-13,0
55,ABCB48480,0,2023-11-13,0
104,ACBY18022,0,2023-11-13,0
476,ADJX46855,0,2023-11-13,0
565,ADRN17806,0,2023-11-13,0
...,...,...,...,...
379,ZXJO91694,7,2023-11-13,2
159,ZZGN21450,4,2023-11-13,2
358,ZZQL21216,5,2023-11-13,2
151,ZZRV30730,7,2023-11-13,2


In [483]:
x = df_stock_filtrado_[df_stock_filtrado_['ml_inventory_id'] == 'DGCL82617']
x

,ml_inventory_id,available_quantity,data,days_available
434,DGCL82617,20,2023-11-13,2


### Calculando se envia

In [484]:
df_orders_filtrado_ = df_orders_filtrado[['ml_inventory_id','item.id', 'ml_code', 'item.seller_sku' ,'ml_sku', 'date_approved', 'sales_quantity', 'total_sales_quantity']]
print(df_orders_filtrado_.shape)
df_orders_filtrado_ = df_orders_filtrado_.drop_duplicates()
print(df_orders_filtrado_.shape)
df_orders_filtrado_.sample()

(128, 8)
(106, 8)


,ml_inventory_id,item.id,ml_code,item.seller_sku,ml_sku,date_approved,sales_quantity,total_sales_quantity
117,WHCM05158,MLB2001122140,MLB2001122140,FULLBASSOCLAPEIXEJESUSPRETO,FULLBASSOCLAPEIXEJESUSPRETO,2023-11-12 18:52:07-04:00,1,1.0


In [485]:
print(df_stock_filtrado_.shape)
print(df_orders_filtrado_.shape)

(596, 4)
(106, 8)


In [486]:
x = df_orders_filtrado_[df_orders_filtrado_['ml_inventory_id'] == 'UTKO14988']
print(x.shape)
x = x.drop_duplicates()
print(x.shape)
x

(5, 8)
(5, 8)


,ml_inventory_id,item.id,ml_code,item.seller_sku,ml_sku,date_approved,sales_quantity,total_sales_quantity
7,UTKO14988,MLB1813063328,MLB1813063328,FULLLIVERPOOLPMLTENC,FULLLIVERPOOLPMLTENC,2023-11-11 21:02:46-04:00,4,9.0
15,UTKO14988,MLB1813063328,MLB1813063328,FULLLIVERPOOLPMLTENC,FULLLIVERPOOLPMLTENC,2023-11-11 23:10:07-04:00,1,9.0
60,UTKO14988,MLB1813063328,MLB1813063328,FULLLIVERPOOLPMLTENC,FULLLIVERPOOLPMLTENC,2023-11-12 11:14:32-04:00,2,9.0
92,UTKO14988,MLB1813063328,MLB1813063328,FULLLIVERPOOLPMLTENC,FULLLIVERPOOLPMLTENC,2023-11-12 14:28:06-04:00,1,9.0
104,UTKO14988,MLB1813063328,MLB1813063328,FULLLIVERPOOLPMLTENC,FULLLIVERPOOLPMLTENC,2023-11-12 16:38:36-04:00,1,9.0


In [487]:
len(df_orders_filtrado_['ml_inventory_id'].value_counts())

35

In [488]:
_df_orders_filtrado_ = df_orders_filtrado_.copy()

indices_recentes = _df_orders_filtrado_.groupby('ml_inventory_id')['date_approved'].idxmax()
indices_recentes

_df_orders_filtrado_ = _df_orders_filtrado_.loc[indices_recentes]
# _df_orders_filtrado_ = _df_orders_filtrado_.drop(['created_at', 'has_stock'], axis=1)

print(df_orders_filtrado_.shape)
print(_df_orders_filtrado_.shape)

(106, 8)
(35, 8)


In [489]:
print(_df_orders_filtrado_.shape)
_df_orders_filtrado_.sample(3)

(35, 8)


,ml_inventory_id,item.id,ml_code,item.seller_sku,ml_sku,date_approved,sales_quantity,total_sales_quantity
28,LESW73286,MLB2024002438,MLB2024002438,FULLERNIEBALLP02146,FULLERNIEBALLP02146,2023-11-12 09:00:57-04:00,1,1.0
108,LUHT78439,MLB2216644379,MLB2216644379,FULLVANDORENSR2615,FULLVANDORENSR2615,2023-11-12 17:11:44-04:00,1,1.0
118,POPO18550,MLB2024265635,MLB2024265635,FULLERNIEBALLP02223,FULLERNIEBALLP02223,2023-11-12 19:00:57-04:00,1,1.0


In [490]:
df_stock_filtrado_

,ml_inventory_id,available_quantity,data,days_available
189,ABCB20467,0,2023-11-13,0
55,ABCB48480,0,2023-11-13,0
104,ACBY18022,0,2023-11-13,0
476,ADJX46855,0,2023-11-13,0
565,ADRN17806,0,2023-11-13,0
...,...,...,...,...
379,ZXJO91694,7,2023-11-13,2
159,ZZGN21450,4,2023-11-13,2
358,ZZQL21216,5,2023-11-13,2
151,ZZRV30730,7,2023-11-13,2


In [507]:
# df_calc = df_stock_filtrado_.merge(_df_orders_filtrado_, on='ml_inventory_id', how='left')
df_unido = pd.merge(df_stock_filtrado_, _df_orders_filtrado_, on='ml_inventory_id', how='left')
df_unido = df_unido.drop(['data', 'item.id', 'ml_code',	'item.seller_sku', 'ml_sku', 'date_approved'], axis=1)
df_unido['sales_quantity'] = df_unido['sales_quantity'].fillna(0)
df_unido['total_sales_quantity'] = df_unido['total_sales_quantity'].fillna(0)

df_unido.shape

(596, 5)

In [508]:
df_unido.sample(15)

,ml_inventory_id,available_quantity,days_available,sales_quantity,total_sales_quantity
21,BGQA78977,0,0,0.0,0.0
391,QXZV73342,0,0,0.0,0.0
2,ACBY18022,0,0,0.0,0.0
449,TNXB03680,53,2,1.0,6.0
427,SPME48894,0,0,0.0,0.0
105,ENML70567,0,0,0.0,0.0
530,XDTG62876,0,0,0.0,0.0
98,ELMX48467,0,0,0.0,0.0
378,PWQU66026,2,2,0.0,0.0
576,YZGH03888,18,2,1.0,8.0


In [509]:
id = 'LESW73286'
x = _df_orders_filtrado_[_df_orders_filtrado_['ml_inventory_id'] == id]
y = df_stock_filtrado_[df_stock_filtrado_['ml_inventory_id'] == id]
z = df_unido[df_unido['ml_inventory_id'] == id]

x

,ml_inventory_id,item.id,ml_code,item.seller_sku,ml_sku,date_approved,sales_quantity,total_sales_quantity
28,LESW73286,MLB2024002438,MLB2024002438,FULLERNIEBALLP02146,FULLERNIEBALLP02146,2023-11-12 09:00:57-04:00,1,1.0


In [510]:
y

,ml_inventory_id,available_quantity,data,days_available
320,LESW73286,10,2023-11-13,2


In [511]:
z

,ml_inventory_id,available_quantity,days_available,sales_quantity,total_sales_quantity
267,LESW73286,10,2,1.0,1.0


In [512]:
_df_orders_filtrado_.sample()

,ml_inventory_id,item.id,ml_code,item.seller_sku,ml_sku,date_approved,sales_quantity,total_sales_quantity
115,SOQF03130,MLB2606294854,MLB2606294854,FULLVANDOREN2CR1025,FULLVANDOREN2CR1025,2023-11-12 18:35:24-04:00,1,4.0


In [513]:
df_stock_filtrado_.sample()

,ml_inventory_id,available_quantity,data,days_available
321,JPOV78690,4,2023-11-13,2


In [514]:
df_unido

,ml_inventory_id,available_quantity,days_available,sales_quantity,total_sales_quantity
0,ABCB20467,0,0,0.0,0.0
1,ABCB48480,0,0,0.0,0.0
2,ACBY18022,0,0,0.0,0.0
3,ADJX46855,0,0,0.0,0.0
4,ADRN17806,0,0,0.0,0.0
...,...,...,...,...,...
591,ZXJO91694,7,2,0.0,0.0
592,ZZGN21450,4,2,0.0,0.0
593,ZZQL21216,5,2,0.0,0.0
594,ZZRV30730,7,2,0.0,0.0


In [522]:
import numpy as np

days = 30

df_unido['media_prod_days_available'] = (df_unido['total_sales_quantity'] / df_unido['days_available'])
df_unido['media_prod_days_available'] = df_unido['media_prod_days_available'].fillna(0)

df_unido['30_send_fulfillment'] = np.ceil((df_unido['total_sales_quantity'] / df_unido['days_available'])* days - df_unido['available_quantity'])
df_unido['30_send_fulfillment'] = df_unido['30_send_fulfillment'].fillna(0)

df_unido['total_send_fulfillment'] = np.ceil((df_unido['total_sales_quantity'] / df_unido['days_available']) - df_unido['available_quantity'])
df_unido['total_send_fulfillment'] = df_unido['total_send_fulfillment'].fillna(0)


df_unido.to_csv('envio.csv')
df_unido.shape

(596, 8)

In [523]:
df_unido

,ml_inventory_id,available_quantity,days_available,sales_quantity,total_sales_quantity,media_prod_days_available,30_send_fulfillment,total_send_fulfillment
0,ABCB20467,0,0,0.0,0.0,0.0,0.0,0.0
1,ABCB48480,0,0,0.0,0.0,0.0,0.0,0.0
2,ACBY18022,0,0,0.0,0.0,0.0,0.0,0.0
3,ADJX46855,0,0,0.0,0.0,0.0,0.0,0.0
4,ADRN17806,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
591,ZXJO91694,7,2,0.0,0.0,0.0,-7.0,-7.0
592,ZZGN21450,4,2,0.0,0.0,0.0,-4.0,-4.0
593,ZZQL21216,5,2,0.0,0.0,0.0,-5.0,-5.0
594,ZZRV30730,7,2,0.0,0.0,0.0,-7.0,-7.0


In [524]:
x = df_unido[df_unido['total_send_fulfillment'] >=1]
x

,ml_inventory_id,available_quantity,days_available,sales_quantity,total_sales_quantity,media_prod_days_available,30_send_fulfillment,total_send_fulfillment
204,JDTL81315,1,2,1.0,4.0,2.0,59.0,1.0
544,XYRN72687,0,1,1.0,1.0,1.0,30.0,1.0
